In [1]:
import sys, os

from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import numpy as np

import data_paths
import cifar10, inception
from inception import transfer_values_cache
from food import food_labels, food_examples

print inception.data_dir
path_graph_def = "classify_image_graph_def.pb"
model_path = os.path.join(inception.data_dir, path_graph_def)
print model_path

import tensorflow as tf

graph = tf.Graph()
with graph.as_default():
    with tf.gfile.FastGFile(model_path, 'rb') as file:
        graph_def = tf.GraphDef()
        graph_def.ParseFromString(file.read())
        tf.import_graph_def(graph_def, name='')

        # Name of the tensor for feeding the input image as jpeg.
tensor_name_input_jpeg = "DecodeJpeg/contents:0"

# Name of the tensor for feeding the decoded input image.
# Use this for feeding images in other formats than jpeg.
tensor_name_input_image = "DecodeJpeg:0"

# Name of the tensor for the resized input image.
# This is used to retrieve the image after it has been resized.
tensor_name_resized_image = "ResizeBilinear:0"

# Name of the tensor for the output of the softmax-classifier.
# This is used for classifying images with the Inception model.
tensor_name_softmax = "softmax:0"

# Name of the tensor for the unscaled outputs of the softmax-classifier (aka. logits).
tensor_name_softmax_logits = "softmax/logits:0"

# Name of the tensor for the output of the Inception model.
# This is used for Transfer Learning.
tensor_name_transfer_layer = "pool_3:0"
# Get the output of the Inception model by looking up the tensor
# with the appropriate name for the output of the softmax-classifier.
y_pred = graph.get_tensor_by_name(tensor_name_softmax)

# Get the unscaled outputs for the Inception model (aka. softmax-logits).
y_logits = graph.get_tensor_by_name(tensor_name_softmax_logits)

# Get the tensor for the resized image that is input to the neural network.
resized_image = graph.get_tensor_by_name(tensor_name_resized_image)

# Get the tensor for the last layer of the graph, aka. the transfer-layer.
transfer_layer = graph.get_tensor_by_name(tensor_name_transfer_layer)

# Get the number of elements in the transfer-layer.
transfer_len = transfer_layer.get_shape()[3]

# Create a TensorFlow session for executing the graph.
in_session = tf.Session(graph=graph)

/Users/ulzee/dev/ml/_data/inception/model/
/Users/ulzee/dev/ml/_data/inception/model/classify_image_graph_def.pb


In [2]:
labels = food_labels()
flattened_examples = []
for lbl in labels:
    examples = food_examples(lbl, load_images=False)
    flattened_examples += [(one, lbl.name, lbl.id) for one in examples]
print len(flattened_examples), flattened_examples[0]
    

31395 ('/Users/ulzee/dev/ml/_data/FOOD256/data/1/1-filled.jpg', 'rice', '1')


In [3]:
from random import shuffle
shuffle(flattened_examples)
print flattened_examples[0]

('/Users/ulzee/dev/ml/_data/FOOD256/data/188/311898-filled.jpg', 'beef in oyster sauce', '188')


In [4]:
set_size = len(flattened_examples)
num_reserved = 2
train_set, test_set = flattened_examples[:-num_reserved], flattened_examples[-num_reserved:]
print len(train_set), len(test_set)

31393 2


In [7]:
print transfer_len, len(labels)
x = tf.placeholder(tf.float32, shape=[None, transfer_len], name='x')
y_true = tf.placeholder(tf.float32, shape=[None, len(labels)], name='y_true')
y_correct = tf.argmax(y_true, axis=1)


2048 256


In [9]:
import prettytensor as pt
x_pretty = pt.wrap(x)

with pt.defaults_scope(activation_fn=tf.nn.relu):
    y_output, loss = x_pretty                                        \
        .fully_connected(size=len(labels), name='PTT_layer_fc1')    \
        .softmax_classifier(num_classes=len(labels), labels=y_true)

global_step = tf.Variable(initial_value=0, name='global_step', trainable=False)
optimizer = tf.train.AdamOptimizer(learning_rate=1e-4).minimize(loss, global_step)
y_guess = tf.argmax(y_output, axis=1)
y_is_correct = tf.equal(y_guess, y_correct)
accuracy = tf.reduce_mean(tf.cast(y_is_correct, tf.float32))

In [10]:
session = tf.Session()
session.run(tf.global_variables_initializer())

In [15]:
def in_create_feed_dict(image_path, image=None):
    if image is not None:
        # Image is passed in as a 3-dim array that is already decoded.
        feed_dict = {tensor_name_input_image: image}

    elif image_path is not None:
        # Read the jpeg-image as an array of bytes.
        image_data = tf.gfile.FastGFile(image_path, 'rb').read()

        # Image is passed in as a jpeg-encoded image.
        feed_dict = {tensor_name_input_jpeg: image_data}

    else:
        raise ValueError("Either image or image_path must be set.")

    return feed_dict

def classify(image_path):
    # Create a feed-dict for the TensorFlow graph with the input image.
    feed_dict = in_create_feed_dict(image_path)

    # Execute the TensorFlow session to get the predicted labels.
    pred = in_session.run(transfer_layer, feed_dict=feed_dict)

    # Reduce the array to a single dimension.
    pred = np.squeeze(pred)

    return pred

In [45]:
# Split the data-set in batches of this size to limit RAM usage.
batch_size = 256

def predict_cls(transfer_values, labels, true_ys):
    # Number of images.
    num_images = len(transfer_values)

    # Allocate an array for the predicted classes which
    # will be calculated in batches and filled into this array.
    guessed_ys = np.zeros(shape=num_images, dtype=np.int)

    # Now calculate the predicted classes for the batches.
    # We will just iterate through all the batches.
    # There might be a more clever and Pythonic way of doing this.

    # The starting index for the next batch is denoted i.
    i = 0

    while i < num_images:
        # The ending index for the next batch is denoted j.
        j = min(i + batch_size, num_images)

        # Create a feed-dict with the images and labels
        # between index i and j.
        feed_dict = {x: transfer_values[i:j],
                     y_true: labels[i:j]}

        # Calculate the predicted class using TensorFlow.
        guessed_ys[i:j] = session.run(y_guess, feed_dict=feed_dict)

        # Set the start-index for the next batch to the
        # end-index of the current batch.
        i = j
        
    # Create a boolean array whether each image is correctly classified.
    correct = np.in1d(true_ys, guessed_ys)

#     return list(correct), list(guessed_ys), list(true_ys)
    return (correct), (guessed_ys), list(true_ys)

def vectorize_id(idnum):
    vec = np.zeros((len(labels)))
    vec[int(idnum) - 1] = 1
    return vec

def predict_cls_test():
    paths, names, ids = zip(*test_set)
    return predict_cls(transfer_values = [classify(one) for one in paths],
                       labels = [vectorize_id(one) for one in ids],
                       true_ys = ids)

print 'Prelim results:', predict_cls_test()

Prelim results: (array([False, False], dtype=bool), array([81, 79]), ['180', '233'])


In [46]:
def print_test_accuracy():
    correct, cls_pred, _ = predict_cls_test()
    acc, num_correct = correct.mean(), correct.sum()
    num_images = len(correct)
    msg = "Accuracy on Test-Set: {0:.1%} ({1} / {2})"
    print(msg.format(acc, num_correct, num_images))

In [47]:
print_test_accuracy()

Accuracy on Test-Set: 0.0% (0 / 2)


In [64]:
def random_batch():
    batch_size = 1
    idx = list(np.random.choice(len(train_set), batch_size, replace=False))
    batch = []
    for ii, item in enumerate(train_set):
        if ii in idx:
            batch.append(item)
    paths, names, ids = zip(*batch)

    x_batch = [classify(one) for one in paths]
    y_batch = [vectorize_id(one) for one in ids]

    return x_batch, y_batch

def optimize(num_iterations):
    for i in range(num_iterations):
        x_batch, y_true_batch = random_batch()

        train_input = {x: x_batch, y_true: y_true_batch}
        i_global, _ = session.run([global_step, optimizer], feed_dict=train_input)

        if (i_global % 1 == 0) or (i == num_iterations - 1):
            batch_acc = session.run(accuracy, feed_dict=train_input)
            msg = "Global Step: {0:>6}, Training Batch Accuracy: {1:>6.1%}"
#             print(msg.format(i_global, batch_acc))
            print(msg.format(i_global, batch_acc))

session.run(tf.global_variables_initializer())
optimize(num_iterations=10000)

Global Step:      1, Training Batch Accuracy:   0.0%
Global Step:      2, Training Batch Accuracy:   0.0%
Global Step:      3, Training Batch Accuracy:   0.0%
Global Step:      4, Training Batch Accuracy:   0.0%
Global Step:      5, Training Batch Accuracy:   0.0%
Global Step:      6, Training Batch Accuracy:   0.0%
Global Step:      7, Training Batch Accuracy:   0.0%
Global Step:      8, Training Batch Accuracy:   0.0%
Global Step:      9, Training Batch Accuracy:   0.0%


KeyboardInterrupt: 